# Use Icechunk with Xarray on ERA5 Data

In this demo we will use Icechunk with Xarray to read and write ERA5 data.

We will start from some data from the [NCAR ERA5 AWS Public Dataset](https://nsf-ncar-era5.s3.amazonaws.com/index.html).

In [1]:
import xarray as xr
import zarr
import dask
import fsspec
import h5py
from dask.diagnostics import ProgressBar

import icechunk
from icechunk import IcechunkStore, StorageConfig

print('xarray:  ', xr.__version__)
print('dask:    ', dask.__version__)
print('zarr:    ', zarr.__version__)
print('icechunk:', icechunk.__version__)

xarray:   0.5.2.dev4594+gd9d6fee2
dask:     0+untagged.8339.gc0f671e
zarr:     3.0.0a7
icechunk: 0.1.0-alpha.1


In [2]:
url = "https://nsf-ncar-era5.s3.amazonaws.com/e5.oper.an.pl/194106/e5.oper.an.pl.128_060_pv.ll025sc.1941060100_1941060123.nc"
%time ds = xr.open_dataset(fsspec.open(url).open(), engine="h5netcdf", chunks={"time": 1})
ds = ds.drop_encoding()

CPU times: user 284 ms, sys: 39.2 ms, total: 323 ms
Wall time: 2.18 s


/srv/conda/envs/icechunk/lib/python3.12/site-packages/xarray/backends/api.py:357: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  var_chunks = _get_chunk(var, chunks, chunkmanager)


In [3]:
print(ds)

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 24, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 192B 1941-06-01 ... 1941-06-01T23:00:00
Data variables:
    PV         (time, level, latitude, longitude) float32 4GB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    utc_date   (time) int32 96B dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.8.1
    CONVERSION_PLATFORM:  Linux r1i4n4 4.12.14-95.51-default #1 SMP Fri Apr 1...
    CONVERSION_DATE:      Wed May 10 06:33:49 MDT 2023
    Conventions:       

### Load Data from HDF5 File

This illustrates how loading directly from HDF5 files on S3 can be slow, even with Dask.

In [4]:
with ProgressBar():
    dsl = ds.load()

[########################################] | 100% Completed | 75.10 ss


### Initialize Icechunk Repo

In [5]:
store = await IcechunkStore.create(
    storage=StorageConfig.s3_from_env(
        bucket="icechunk-test",
        prefix="ryan/icechunk-tests-era5-4"),
    mode="w"
)
store

In [6]:
store.branch, store.snapshot_id

('main', 'NFT4V6D9HTP7YYVPFQCG')

### Store Data To Icechunk

We specify encoding to set both compression and chunk size.

In [7]:
encoding = {
    "PV": {
        "codecs": [zarr.codecs.BytesCodec(), zarr.codecs.ZstdCodec()],
        "chunks": (1, 1, 721, 1440)
    }
}

Note that Dask is not required to obtain good performance when reading and writing. Zarr and Icechunk use multithreading and asyncio internally.

In [8]:
%time dsl.to_zarr(store, zarr_format=3, consolidated=False, encoding=encoding)

CPU times: user 41.9 s, sys: 1.53 s, total: 43.4 s
Wall time: 20.5 s


In [9]:
await store.commit("wrote data")

'FVSG2XNYSF0475HP8DKG'

### Read Data Back

In [10]:
%time dsic = xr.open_dataset(store, consolidated=False, engine="zarr")

CPU times: user 14.8 ms, sys: 428 μs, total: 15.3 ms
Wall time: 50.6 ms


In [11]:
print(dsic)

<xarray.Dataset> Size: 4GB
Dimensions:    (longitude: 1440, time: 24, latitude: 721, level: 37)
Coordinates:
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 192B 1941-06-01 ... 1941-06-01T23:00:00
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    PV         (time, level, latitude, longitude) float32 4GB ...
    utc_date   (time) int32 96B ...
Attributes:
    CONVERSION_DATE:      Wed May 10 06:33:49 MDT 2023
    CONVERSION_PLATFORM:  Linux r1i4n4 4.12.14-95.51-default #1 SMP Fri Apr 1...
    Conventions:          CF-1.6
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NCO:                  netCDF Operators version 5.0.3 (Homepage = http://n...
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    NETCDF_CONVERSION:    CISL RDA:

In [12]:
%time dsic.PV[0, 0, 0, 0].values

CPU times: user 9.18 ms, sys: 7.07 ms, total: 16.2 ms
Wall time: 79.6 ms


array(0.00710905, dtype=float32)

As with writing, Dask is not required for performant reading of the data.
In this example we can load the entire dataset (nearly 4GB) in 8s. 

In [13]:
%time _ = dsic.load()

CPU times: user 8.08 s, sys: 1.93 s, total: 10 s
Wall time: 9.65 s
